In [1]:
import warnings 
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from cleantext import clean

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

### Uploading data

In [2]:
train = pd.read_csv('public_data/train_data.csv')
train_solution = pd.read_csv('public_data/train_solution.csv')
train['category'] = train_solution['category']
test = pd.read_csv('public_data/test_data.csv')
subm = pd.read_csv('public_data/sample_submission.csv')

Также загрузим дополнительные данные из телеграма (парсинг) и большой датасет из kaggle с текстами разной тематики.

In [3]:
ad = pd.read_csv('public_data/tg.csv').dropna().drop(columns=['Unnamed: 0'])
bloggers = pd.read_csv('public_data/blogtext.csv')

В датасете kaggle есть темы, которые подходят нам, поэтому срежем объекты по темам Fashion и Communications-Media, и добавим их для расширения выборки.

In [4]:
fashion = pd.DataFrame(bloggers[(bloggers['topic'] == 'Fashion') | 
                               (bloggers['topic'] == 'Communications-Media')]['text']).rename(columns={'text': 'message'})
fashion['category'] = 2 

In [5]:
bloggers.shape

(681284, 7)

Склеим все данные для трейна.

In [6]:
train_tg = pd.concat([train, ad, fashion], ignore_index=True, axis=0)
y_train_tg = train_tg.category

In [7]:
train_tg.shape

(55365, 3)

Посмотрим на баланс классов

In [8]:
train_tg.category.value_counts() / train_tg.shape[0]

2.0    0.491863
0.0    0.334634
1.0    0.173503
Name: category, dtype: float64

### TF-IDF

Сделаем самое банальное: обучим логрег на tf-idf представлениях предложений:

In [9]:
sw = stopwords.words('english')

def my_tok_and_clean(x):
    a = word_tokenize(clean(x, no_urls=True, no_digits=True, no_punct=True, no_line_breaks=True, no_numbers=True,
             no_emoji=True))
    return [w for w in a if w not in sw]

In [10]:
vectorizer = TfidfVectorizer(tokenizer=my_tok_and_clean, max_features=5000, stop_words=sw, ngram_range=(1, 1))
tfidf_train = vectorizer.fit_transform(train_tg['message'])
tfidf_test = vectorizer.transform(test['message'])

In [11]:
np.mean(cross_val_score(LogisticRegression(random_state=10), tfidf_train, 
                        y_train_tg, cv=5, scoring='accuracy'))

0.915163009121286

Удивительно, но такой "тупой" подход выдает 0.82-0.83 на тесте.

### Saving test predictions for submossion

In [ ]:
result = subm
result['category'] = final_model.predict(tfidf_test.toarray())
result.to_csv('my_submission.csv')